In [1]:
import pandas as pd
import plotnine as p9
import numpy as np
import os

In [4]:
path = 'J:\\BoitePos\\Defacto_Scoring\\Orsay_Promo_salesforce10032020'
os.chdir(path)
df_to_fill = pd.read_csv('Orsay_Promo_salesforce.csv',
                         delimiter = ';',
                         decimal = ',')

In [5]:
df_to_fill.shape

(561394, 3)

In [6]:
path = 'J:\BoitePos\\Julien_Exchange'
os.chdir(path)
df_history = pd.read_csv('JULIEN_2020-03-04T15_46_502020-03-04 15-47-54.csv',
                         delimiter = ';')


In [7]:
df_merged = pd.merge(df_to_fill,
                     df_history[['CLBC_SALESFORCE_ID', 
                                 'CTRY_CODE',
                                 'CLUB_CARD_LAST_DATE',
                                 'CLUB_CARD_INIT_DATE']].drop_duplicates(),
                     left_on = 'SFID',
                     right_on = 'CLBC_SALESFORCE_ID',
                     how='left'
                     )

In [8]:
df_merged = (df_merged.rename(columns={'CTRY_CODE': 'country',
                                      'CLUB_CARD_LAST_DATE': 'last_receipt_date',
                                      'CLUB_CARD_INIT_DATE': 'first_receipt_date'})
                      .drop(columns={'CLBC_SALESFORCE_ID'})
            )

In [ ]:
df_merged = (df_merged
            .assign(last_date = lambda d: pd.to_datetime(d.last_receipt_date))
            .assign(first_date = lambda d: pd.to_datetime(d.first_receipt_date))
            )

In [11]:
number_WH_country = {
    'AT': 22100,
    'CH': 0,
    'CZ':23800,
    'HU':11050,
    'DE':68000,
    'PL':61200,
    'SK':7990,
    'RO':5610,
    'HR':3910,
    'RS':3060,
    'SI':4080
                    }

In [14]:
(df_merged.loc[df_merged.country == 'DE']
.sort_values(by='Score', ascending=False)
.iloc[[v for k,v in number_WH_country.items() if k == 'DE'][0]]
).Score

0.6582399999999999

In [27]:
df_final = pd.DataFrame()
for i in df_merged.country.dropna().unique():
    df_curr = df_merged.loc[df_merged.country == i]
    print(i + ': ' + str(df_curr
     .sort_values(by='Score', ascending=False)
     .iloc[[v for k,v in number_WH_country.items() if k == i][0]].Score.round(decimals=3))
    )

    

PL: 0.647
SK: 0.625
RS: 0.495
DE: 0.658
AT: 0.632
CZ: 0.639
RO: 0.653
HU: 0.647
HR: 0.623
SI: 0.597
CH: 0.96


In [28]:
(df_merged
 .loc[df_merged.country == 'DE']
 .loc[lambda d: d.Score > 0.658]
)

SFID         ScoringIdentifier     Score country  \
9       0035800000HQNYKAA5  Reactivation_March_ 2020  0.794845      DE   
15      0035800001WDeqtAAD  Reactivation_March_ 2020  0.758960      DE   
16      0035800000HPpdRAAT  Reactivation_March_ 2020  0.750462      DE   
17      0031i000004UV9nAAG  Reactivation_March_ 2020  0.876596      DE   
19      0035800000HYAfFAAX  Reactivation_March_ 2020  0.736785      DE   
...                    ...                       ...       ...     ...   
585199  0035800000HQolfAAD  Reactivation_March_ 2020  0.943089      DE   
585207  0035800000HTMXWAA5  Reactivation_March_ 2020  0.671006      DE   
585225  0035800000HS5xLAAT  Reactivation_March_ 2020  0.721893      DE   
585255  0035800000jOsazAAC  Reactivation_March_ 2020  0.813431      DE   
585258  00358000015sIEIAA2  Reactivation_March_ 2020  0.815903      DE   

       last_receipt_date first_receipt_date  
9             2019-05-25         2009-11-28  
15            2019-07-02         2018-06-04  
16            2019-06-29         2009-10-12  
17            2019-06-29         2018-09-13  
19            2019-05-02         2016-02-20  
...                  ...                ...  
585199        2019-07-20         2009-12-04  
585207        2020-02-26         2015-09-12  
585225        2019-06-05         2011-06-15  
585255        2020-03-02         2015-09-17  
585258        2020-02-01         2017-09-16  

[68068 rows x 6 columns]